<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado desde 2018-1 al 2023-1 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos

1. [Iterables](#Iterables)
    1. [Iterar sobre estructuras de datos](#Iterar-sobre-estructuras-de-datos)
    2. [Forma básica de hacer una estructura iterable](#Forma-básica-de-hacer-una-estructura-iterable)
1. [Generadores](#Generadores)
    1. [Ejemplos](#Ejemplos)
    2. [Otra forma de hacer iterable una estructura propia](#Otra-forma-de-hacer-iterable-una-estructura-propia)
    3. [También podemos interactuar con la función generadora enviando mensajes](#También-podemos-interactuar-con-la-función-generadora-enviando-mensajes)

# Iterables

Esta semana revisaremos técnicas que diversos lenguajes, entre ellos Python, proveen para recorrer estructuras de datos de manera sencilla y genérica. Veremos como implementar estos elementos y aplicar a estructuras existentes, o a estructuras creadas por nosotros.

## Iterar sobre estructuras de datos 

En muchas ocasiones implementaremos estructuras de datos en que resulta natural la noción de que pueden ser recorridas. Este es el caso para estructuras que hemos visto como listas, tuplas, *sets* y diccionarios. Nos gustaría que, cuando hagamos nuestras propias estructuras, algunas también puedan ser recorridas utilizando `for`, como lo haríamos con una lista. Para esto hay que entender dos conceptos claves: **iterable** e **iterador**.

Un **iterable** es cualquier **objeto sobre el cual se puede iterar**. Un **iterable** podría aparecer al lado derecho de un `for` (`for i in iterable:`). Estructuras *built-ins* como *sets*, listas, diccionarios y *deques*, son **iterables**.

Un **iterable** siempre tiene **implementado el método** **`__iter__()`**. Se puede iterar todas las veces que uno quiera sobre un iterable, como es el caso de las listas. No es necesario que el objeto iterable se pueda indexar. Por ejemplo, los *sets* no se indexan, pero sí podemos iterar sobre ellos, como en el siguiente ejemplo:

In [1]:
# Un set no es indexable, pero sí es iterable
conjunto = {1, 3, 4, 6}

for i in conjunto:
    print(i, end=" ")

1 3 4 6 

**Nota**: usamos `print(i, end=" ")` donde `end=" "` para que el fin de un `print` sea solo un espacio y no un salto de línea (`\n`). Así el print va hacia la derecha en vez de un número por línea.

Por otra parte, un **iterador** es un **objeto que itera sobre un iterable**, y es el objeto retornado por el método `__iter__()`. Este objeto iterador implementa el método `__next__()`, que nos retorna uno a uno los elementos de la estructura cada vez que se invoca a esta función. Cuando no quedan objetos por recorrer el iterador **debe** levantar una excepción de tipo `StopIteration`. 

In [2]:
# iter(conjunto) nos entrega un objeto que itera sobre ese conjunto
conjunto = {1, 3, 4, 6}
iterador = iter(conjunto)  # Esto es lo mismo que conjunto.__iter__()
print(type(iterador))

# Ahora vamos a invocar a next para que el iterador nos entregue el siguiente valor del iterable
print(next(iterador))      # Esto es lo mismo que iterador.__next__()
print(next(iterador))
print(next(iterador))

<class 'set_iterator'>
1
3
4


Si al iterador le pedimos más elementos de los que tiene la estructura, levantará una excepción de tipo `StopIteration`.

In [3]:
print(next(iterador))
print(next(iterador))

6


StopIteration: 

**Recordar** 

> Un iterable debe tener el método `__iter__` implementado, y debe retornar **siempre** un iterador. Por su parte, un iterador es un objeto que tiene el método `__next__` implementado, es decir puedo hacer `next(iterador)` y esto retornará un **valor**.

### Forma básica de hacer una estructura iterable

Ahora veremos la forma más básica de crear un iterable, que es creando una **clase iteradora**. Hay más formas de hacer que una estructura sea iterable sin crear una nueva clase, entre ellas una que veremos [más adelante](#Otra-forma-de-hacer-iterable-una-estructura-propia).

Empezamos creando una clase, y datos con los que vamos a trabajar.

In [4]:
class Nodo:
    
    def __init__(self, valor, siguiente):
        # Cada nodo contiene un valor...
        self.valor = valor
        # ... y referencia al siguiente Nodo
        self.siguiente = siguiente
    
    def __repr__(self):
        return f"{self.valor}"

Creamos ahora la clase `Iterable` que implementará el método `__iter__`. 

In [5]:
class Iterable:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        return Iterador(self.objeto)

Ahora creamos la clase `Iterador` que será la encargada de iterar sobre el iterable. Esta clase debe implementar el método `__next__` que retornará los valores.

El método `__iter__` sólo debe retornar `self`. Es posible no implementarlo, pero no será posible obtener el iterador `iterador = iter(iterable)` y luego usar el iterador directamente en el `for`.

En general, los iteradores no se pueden reiniciar.

In [6]:
class Iterador:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __iter__(self): 
        return self
    
    def __next__(self):
        if self.iterable is None:
            # Levantamos una excepción del tipo StopIteration
            # con el mensaje "Llegamos al final".
            raise StopIteration("Llegamos al final")
        else:           
            valor = self.iterable
            self.iterable = self.iterable.siguiente
            return valor

Ahora vamos a generar nuestro iterable con los datos de una lista ligada.

In [7]:
datos = Nodo(1, Nodo(2, Nodo(3, Nodo(4, Nodo(5, None)))))
iterable = Iterable(datos)
for i in iterable:
    print(i, end=" ")

1 2 3 4 5 

Recordemos que un iterable se puede iterar las veces que uno quiera. Si volvermos a hacer el `for`:

In [8]:
for i in iterable:
    print(i, end=" ")

1 2 3 4 5 

Por otro lado, si usamos un **iterador**, este solo funcionará una vez.

In [9]:
iterador = Iterador(datos)
# Primer intento
for i in iterador:
    print(i, end=" ")
    
# Segundo intento con el mismo iterador
for i in iterador:
    print(i, end=" ")

1 2 3 4 5 

Por este motivo, cada vez que queramos tener una conjunto de datos iterable, tenemos que construir un `Iterable` cuyo método `__iter__` retorne un nuevo `Iterador`. De este modo, cada vez que hagamos `for`, se retorna un nuevo `Iterador`.

Ahora, vemos un nuevo caso:  **si el iterador no implementa `__iter__`**

In [10]:
class IterableMalo:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        return IteradorMalo(self.objeto)

    
class IteradorMalo:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __next__(self):
        if self.iterable is None:
            raise StopIteration("Llegamos al final")
        else:           
            valor = self.iterable
            self.iterable = self.iterable.siguiente
            return valor

In [11]:
iterable = IterableMalo(datos)
iterador = iter(iterable)
for i in iterador:
    print(i, end=" ")

TypeError: 'IteradorMalo' object is not iterable

Esto ocurre porque `IteradorMalo` no tiene `__iter__` y por lo tanto, para el `for` esta clase no es **iterable**.

Cómo se mencionó anteriormente, un iterador solo se puede recorrer una vez. La ventaja de esto es que podemos iterrumpir el recorrido y luego continuar desde el punto en que lo dejamos:

In [12]:
iterable = Iterable(datos)
iterador = iter(iterable)
for i in iterador:
    print(i, end=" ")
    if i.valor >= 3:
        break

1 2 3 

In [13]:
for i in iterador:
    print(i, end=" ")

4 5 

Para empezar nuevamente a iterar, debemos **obtener otro iterador**. El código de abajo funciona, debido a que cuando se invoca otra vez la función `__iter__` del iterable debido al `for`, se retorna un **nuevo iterador**.

In [14]:
for i in iterable:
    print(i, end=" ")

1 2 3 4 5 

Cada iterador tiene su propia memoria, la cual **no depende del iterable**. Para verlo, creamos dos iteradores:

In [15]:
iterador_1 = iter(iterable)
iterador_2 = iter(iterable)

Utilizamos el primero sólo tres veces:

In [16]:
for i in iterador_1:
    print(i, end=" ")
    if i.valor >= 3:
        break

1 2 3 

Y el segundo iterador recorre la colección otra vez, independiente de lo recorrido por el primero.

In [17]:
for i in iterador_2:
    print(i, end=" ")

1 2 3 4 5 

Pero si volvemos a recorrer la estructura con el primer iterador (que sólo usamos tres veces), continuaremos desde donde lo dejamos:

In [18]:
for i in iterador_1:
    print(i, end=" ")

4 5 

Por último, una vez que el iterador agotó la estructura de datos no lo podemos utilizar de nuevo.

In [19]:
for i in iterador_1:
    print(i, end=" ")

# Generadores

Los **generadores** son un caso especial de los **iteradores**. Los generadores nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura especial, evitando el uso innecesario de memoria. 

Una vez que terminamos de iterar sobre un generador, el generador desaparece. Esto es muy útil cuando queremos realizar cálculos sobre secuencias de números que sólo nos sirven para ese cálculo en particular. La sintaxis para crear generadores es muy parecida a la comprensión de listas, sólo que en vez de paréntesis cuadrados `[` `]` usamos paréntesis normales `(` `)`.

Por ejemplo, crearemos un generador para los números pares del 0 al 18:

In [20]:
generador_pares = (2 * i for i in range(10)) # Por el sólo hecho de usar paréntesis estamos creando un generador.

In [21]:
print(type(generador_pares))

<class 'generator'>


Ahora vamos a mostrar lo que nos entrega con un `for`. Esto es posible ya que **los generadores implementan `__iter__`** retornando `self`.

In [22]:
for i in generador_pares:
    print(i, end=" ")

0 2 4 6 8 10 12 14 16 18 

Nuevamente, como los generadores son un caso especial de iteradores, estos no se pueden usar otra vez, sino que tendremos que crear otro nuevo.

In [23]:
for i in generador_pares:
    print(i, end=" ")

Recordar también que se puede usar `next`:

In [24]:
generador_pares = (2 * i for i in range(10))
print(next(generador_pares))
print(next(generador_pares))

0
2


Veamos cuánta memoria ocupa nuestro generador, *versus* una lista que contiene todos los resultados. Para ello ocuparemos la función [`getsizeof`](https://docs.python.org/3/library/sys.html#sys.getsizeof).

In [25]:
from sys import getsizeof

In [26]:
generador_pares = (2 * i for i in range(10))
lista_pares = [2 * i for i in range(10)]

print("Bytes del generador:", getsizeof(generador_pares))
print("Bytes de la lista:", getsizeof(lista_pares))

Bytes del generador: 104
Bytes de la lista: 184


Esta diferencia es más grande si aumentamos la cantidad de resultados:

In [27]:
generador_pares = (2 * i for i in range(10 ** 6))
lista_pares = [2 * i for i in range(10 ** 6)]

print("Bytes del generador:", getsizeof(generador_pares))
print("Bytes de la lista:", getsizeof(lista_pares))

Bytes del generador: 104
Bytes de la lista: 8448728


Concluimos que una ventaja de los generadores es que consumen mucho menos memoria, ya que **generan** cada nuevo elemento de la secuencia cuando se le solicita, y no mantienen todos los elementos de la secuencia en memoria. Es particularmente útil cuando queremos leer archivos con muchos datos. En lugar de usar una instrucción `archivo.readlines()` para leer todos los datos de una sola vez en memoria (supongamos que queremos leer un archivo de 1GB), podemos abrir el archivo, y usar un generador para extraer una línea a la vez y así evitar llenar la memoria. Precisamente el siguiente tema **Funciones generadores** permite construir una función que abra un archivo, y entregue una línea del archivo cada vez.

## Funciones Generadoras

Las funciones en Python también tienen la posibilidad de funcionar como generadores, con la sentencia `yield`. El *statement* `yield` es un análogo a `return`, con ciertas diferencias. Por un lado, `yield` se encarga de retornar el valor indicado, pero también se asegura que en la próxima llamada a la función, la ejecución parta desde donde se dejó antes. 

En otras palabras, trabajamos con una función que una vez que entrega un valor a través de `yield`, está cediendo el control sólo en forma temporal, asumiendo que pronto será utilizada nuevamente para generar más valores.

Creemos nuestra primera función generadora de números decrecientes:

In [28]:
def conteo_decreciente(n):
    print(f"Contando en forma decreciente desde {n}")
    while n > 0:
        yield n
        n -= 1

Vemos que cuando se llama a la función generadora, esta no ejecuta nada:

In [29]:
x = conteo_decreciente(10)

Esto se debe a que cuando se invoca la función generadora, esta retorna un **generador**. Luego, `x` es un generador de números desde el 10 hasta el 1.

In [30]:
print(type(x))

<class 'generator'>


Podemos usar el generador directamente en un `for`, ya que como vimos estos implementan `__iter__` devolviendo `self`.

In [31]:
for number in x:
    print(number)

Contando en forma decreciente desde 10
10
9
8
7
6
5
4
3
2
1


También se puede usar `next`:

In [32]:
x = conteo_decreciente(5)
print(next(x))

Contando en forma decreciente desde 5
5


### Ejemplos

Veamos un ejemplo de una función generadora de números de Fibonacci:

In [33]:
def fibonacci():
    a, b = 0, 1
    while True: # Notar que este generador nunca "se agota"
        yield b
        a, b = b, a + b

generador_fibonacci = fibonacci()

# Imprimimos los primeros 5 elementos
for i in range(5):
    print(next(generador_fibonacci))

1
1
2
3
5


También veamos un ejemplo de que las funciones generadoras pueden operar con otras colecciones, como listas:

In [34]:
def maximo_acumulativo(valores):
    """Retorna el máximo visto hasta ahora en una colección de valores."""
    max_ = float('-inf')
    for valor in valores:
        max_ = max(valor, max_)
        yield max_
        
lista = [1, 10, 14, 7, 9, 12, 19, 33]

for i in maximo_acumulativo(lista):
    print(i)

1
10
14
14
14
14
19
33


### Otra forma de hacer iterable una estructura propia

Si tenemos una estructura de datos propia, podemos usar una función generadora en `__iter__` en vez de crear nuestra propia clase iteradora. Para ilustrarlo usaremos el mismo ejemplo del principio:

In [35]:
class Nodo:
    
    def __init__(self, valor, siguiente):
        self.valor = valor
        self.siguiente = siguiente
    
    def __repr__(self):
        return f"{self.valor}"

In [36]:
class Iterable:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        nodo_actual = self.objeto
        while nodo_actual:
            yield nodo_actual
            nodo_actual = nodo_actual.siguiente

In [37]:
datos = Nodo(1, Nodo(2, Nodo(3, Nodo(4, Nodo(5, None)))))
iterable = Iterable(datos)

In [38]:
for x in iterable:
    print(x, end=" ")

1 2 3 4 5 

In [39]:
iterador = iter(iterable)
print(type(iterador))

<class 'generator'>


### También podemos interactuar con la función generadora enviando mensajes 

El método `send()` permite enviar un valor hacia el generador, lo que significa que la expresión `yield` lo recibirá. El valor enviado puede ser usado para asignarlo a otra variable, por ejemplo `v = yield value` guardará en la variable `v` el valor enviado con `send()`.

Veamos un ejemplo de una función generadora que entrega números que se incrementan según lo que le es enviado mediante `send`:

In [40]:
def funcion_generadora_send():
    contador = 0
    while True:
        valor_recibido = yield contador
        print("Hemos recibido {}".format(valor_recibido))
        if valor_recibido is None:  # Consideraremos 0 si nos llega un None
            valor_recibido = 0
        print("Sumaremos {} a nuestro contador".format(valor_recibido))
        contador += valor_recibido  # Sumamos el valor recibido al contador que llevamos

In [41]:
generador_send = funcion_generadora_send()

Lo primero en lo que nos tenemos que fijar es que debemos avanzar hasta `yield` antes de poder enviar valores. Es decir, la primera vez no podremos enviar nada, sólo usar `next`:

In [42]:
generador_send.send(5)

TypeError: can't send non-None value to a just-started generator

Ya hecho el primer `yield`, podemos enviarle valores a la función generadora:

In [43]:
next(generador_send)

0

In [44]:
generador_send.send(5)

Hemos recibido 5
Sumaremos 5 a nuestro contador


5

Hacer `next`, es equivalente a hacer `send` de `None`.

In [45]:
next(generador_send)

Hemos recibido None
Sumaremos 0 a nuestro contador


5

In [46]:
generador_send.send(10)

Hemos recibido 10
Sumaremos 10 a nuestro contador


15

**Ahora, con los contenidos que ya has estudiado, puedes practicar el uso de iterables (e iteradores) y generadores en la primera sección de los ejercicios propuestos. Recuerda repasar las definiciones de los conceptos (como los métodos que necesitas implementar en iterables e iteradores) y el funcionamiento de los generadores.**